# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [4]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


In [17]:
#Q1  What is the sum of the outputs of the generator for limit = 5?
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = list(square_root_generator(limit))

q1 = 0
for sqrt_value in generator:
    q1+=sqrt_value
print(q1)


8.382332347441762


In [21]:
#Q2  What is the 13th number yielded
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = list(square_root_generator(limit))

print(generator[12])

3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [ ]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [21]:
#Load first generator
import dlt

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

data = list(people_1())

pipeline = dlt.pipeline(pipeline_name='people_data',
                        destination='duckdb', 
                        dataset_name='people_info')
info = pipeline.run(data,
                    table_name = "people",
                    write_disposition="replace")

print(info)


Pipeline people_data load step completed in 0.23 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_info
The duckdb destination used duckdb:///e:\Data Engineering\data-engineering-zoomcamp\workshop_1_data_ingestion\people_data.duckdb location to store data
Load package 1708254778.0607378 is LOADED and contains no failed jobs


In [22]:
#Append second generator

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

data = list(people_2())

pipeline = dlt.pipeline(pipeline_name='people_data',
                        destination='duckdb', 
                        dataset_name='people_info')
info = pipeline.run(data,
                    table_name = "people",
                    write_disposition="append")

print(info)


Pipeline people_data load step completed in 0.23 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_info
The duckdb destination used duckdb:///e:\Data Engineering\data-engineering-zoomcamp\workshop_1_data_ingestion\people_data.duckdb location to store data
Load package 1708254795.0543318 is LOADED and contains no failed jobs


In [25]:
pipeline = dlt.pipeline(
    pipeline_name="people_data",
    destination="duckdb",
    dataset_name="people_info",
    full_refresh=True
)
with pipeline.sql_client() as client:
    with client.execute_query(
        'SELECT * FROM people_info.people'
    ) as table:
        # calling `df` on a cursor, returns the data as a DataFrame
        people_df = table.df()

people_df

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708254778.0607378,TTPaMg9gK5J9NA,None
1,2,Person_2,27,City_A,1708254778.0607378,8U7zpIPC/bT6LA,None
2,3,Person_3,28,City_A,1708254778.0607378,J4clbcdKaMWDSw,None
3,4,Person_4,29,City_A,1708254778.0607378,ZINvTO8wq5wI6Q,None
4,5,Person_5,30,City_A,1708254778.0607378,FAe+V6EUDReCtQ,None
5,3,Person_3,33,City_B,1708254795.0543318,0UiDCR5zcCrG6g,Job_3
6,4,Person_4,34,City_B,1708254795.0543318,qywHN0U4Yrs34w,Job_4
7,5,Person_5,35,City_B,1708254795.0543318,vjUV4ya3nd8+/Q,Job_5
8,6,Person_6,36,City_B,1708254795.0543318,L7d+yIUbj5LEuA,Job_6
9,7,Person_7,37,City_B,1708254795.0543318,We6rbD2nd8/Hfw,Job_7


In [27]:
#Q3  Append the 2 generators. After correctly appending the data, calculate the sum of all ages of people.
pipeline = dlt.pipeline(
    pipeline_name="people_data",
    destination="duckdb",
    dataset_name="people_info",
    full_refresh=True
)
with pipeline.sql_client() as client:
    with client.execute_query(
        'SELECT sum(age) FROM people_info.people'
    ) as table:
        # calling `df` on a cursor, returns the data as a DataFrame
        q3 = table.df()

q3

,sum(age)
0,353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [18]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

UsageError: Line magic function `%%capture` not found.


In [28]:
#Load first generator
import dlt

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

data = list(people_1())

pipeline = dlt.pipeline(pipeline_name='people_data',
                        destination='duckdb', 
                        dataset_name='people_info')
info = pipeline.run(data,
                    table_name = "people_new",
                    write_disposition="replace")

print(info)


Pipeline people_data load step completed in 0.31 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_info
The duckdb destination used duckdb:///e:\Data Engineering\data-engineering-zoomcamp\workshop_1_data_ingestion\people_data.duckdb location to store data
Load package 1708255055.830727 is LOADED and contains no failed jobs


In [31]:
#Append second generator

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

data = list(people_2())

pipeline = dlt.pipeline(pipeline_name='people_data',
                        destination='duckdb', 
                        dataset_name='people_info')
info = pipeline.run(data,
                    table_name = "people_new",
                    write_disposition="merge",
                    primary_key="ID"
)

print(info)


Pipeline people_data load step completed in 0.55 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_info
The duckdb destination used duckdb:///e:\Data Engineering\data-engineering-zoomcamp\workshop_1_data_ingestion\people_data.duckdb location to store data
Load package 1708255600.4423137 is LOADED and contains no failed jobs


In [32]:
pipeline = dlt.pipeline(
    pipeline_name="people_data",
    destination="duckdb",
    dataset_name="people_info",
    full_refresh=True
)
with pipeline.sql_client() as client:
    with client.execute_query(
        'SELECT * FROM people_info.people_new'
    ) as table:
        # calling `df` on a cursor, returns the data as a DataFrame
        people_new_df = table.df()

people_new_df

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708255055.830727,+cb8D4P0OP8Q3A,None
1,2,Person_2,27,City_A,1708255055.830727,PBRT2uHEh0ovRQ,None
2,5,Person_5,35,City_B,1708255600.4423137,Q8NHN92Jauba8A,Job_5
3,7,Person_7,37,City_B,1708255600.4423137,4wwMd4p8VRth3Q,Job_7
4,4,Person_4,34,City_B,1708255600.4423137,7gK7PxmM2/nZeg,Job_4
5,8,Person_8,38,City_B,1708255600.4423137,8Y3+y4KKZfMH0A,Job_8
6,3,Person_3,33,City_B,1708255600.4423137,sy0cNZnupOBsCA,Job_3
7,6,Person_6,36,City_B,1708255600.4423137,5iiVdTAoT5Z6iA,Job_6


In [33]:
#Q4  Merge the 2 generators using the ID column. Calculate the sum of ages of all the people loaded as described above
pipeline = dlt.pipeline(
    pipeline_name="people_data",
    destination="duckdb",
    dataset_name="people_info",
    full_refresh=True
)
with pipeline.sql_client() as client:
    with client.execute_query(
        'SELECT sum(age) FROM people_info.people_new'
    ) as table:
        # calling `df` on a cursor, returns the data as a DataFrame
        q4 = table.df()

q4

,sum(age)
0,266.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX